In [1]:
import pandas as pd

### Задание 1
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# Немного ненужного синтаксического сахара :-)
# https://codewars.com/users/tadoran/completed_solutions
import functools
def format_duration(seconds):
    if not seconds: return 'now'
    
    seq = [1, 1, 60, 60, 24, 30, 12]
    vars = [0] * (len(seq)-1)
    for i in range(1,len(seq)):
        vars[i-1] = functools.reduce(lambda a,b: a*b, seq[0:i+1])
    vars_vals = ([
                    seconds           // vars[5],
                    seconds % vars[5] // vars[4],
                    seconds % vars[4] // vars[3],
                    seconds % vars[3] // vars[2],
                    seconds % vars[2] // vars[1], 
                    seconds % vars[1] // vars[0]
                ])
    vars_txt  = ["year", "month", "day", "hour", "minute", "second"]
    
    txt_list = []
    for i, val in enumerate(vars_vals):
        if val == 1:
            txt_list += [str(val) + " " + vars_txt[i]]
        elif val > 1:
            txt_list += [str(val) + " " + vars_txt[i] + "s"]
    if len(txt_list) > 1:
        return ", ".join(txt_list[:-1]) + " and " + txt_list[-1]
    else:
        return ", ".join(txt_list)

In [4]:
user_info = ratings.groupby('userId').agg({'timestamp':['min','max'], 'userId':"count"})
user_info["lifetime"] = user_info["timestamp"]["max"] - user_info["timestamp"]["min"]
avg_lifetime = int(user_info[ user_info["userId"]["count"] > 100 ]["lifetime"].mean())
print(f"Average lifetime for user with > 100 reviews: {format_duration(avg_lifetime)}")

Average lifetime for user with > 100 reviews: 1 year, 5 months, 7 days, 17 hours, 22 minutes and 47 seconds


### Задание 2

#### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:
 1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
 1. аналогичную таблицу по типам выручки с указанием адреса клиента.  
 
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
revenue = rzd.merge(auto, how='outer', on="client_id").merge(air, how='outer', on="client_id")

In [9]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [10]:
full_table = client_base.merge(revenue, on="client_id", how="outer")

In [11]:
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [12]:
full_table

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


### Задание 3.
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
 1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
 1. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
 1. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Для связки с визитами / покупками, для каждой записи покупки берем время, добавляем какой-то адекватный задаче интервал времени вокруг визита (напр, 10 минут до визита, 10 мин – после).  
Делаем запрос к таблице с координатами, смотрим кол-во записей, входящих в интервал времени для соответсвующего user_id.  
Для целей грубого анализа, можем усреднить полученные дынные по широте и долготе за какой-то период (предположим, за 20 минут изменения - не слишком значительны).  
В идеале, данные бы кластерезировать (напр., пользователь за 20 минут поездки на метро изменил местоположение и есть несколько мест). Тогда для одного визита можно сделать несколько записей, например, для расчета вектора перемещений.

По координатам можем расчитать, например, тип местности (загород / дорога / метро / дом (спальный район) / работа (центр или большое кол-во дневного времени в одной зоне) и т.п
Из данных можно попробовать делать предположения о том, в какой ситуации обычно пользователи совершают покупку.
Также, можем сегментировать клиентов по регионам \ городам для более точной контекстной рекламы.
По геометкам из определенных мест можем попробовать делать выводы об интересах пользователя, чтобы опять-же, давать ему более релевантные предложения / рекламу.